<a href="https://colab.research.google.com/github/krahul2024/machine-learning/blob/main/projects/next_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import requests

file_url = 'https://raw.githubusercontent.com/krahul2024/machine-learning/main/projects/Text-Datasets/paul_graham_essays.txt'
content = requests.get(file_url).text
len(content)

3023219

In [2]:
vocab = sorted(list(set(content)))
vocab_size, total_chars = len(vocab), len(content)
vocab_size

106

In [3]:
#@title Imports related to the model
import tensorflow as tf
from tensorflow import keras

In [ ]:
char_index = dict((c, i) for i, c in enumerate(vocab))
char_index

In [6]:
input_size = 40
steps = 3
x , y = [], []
for i in range(0, total_chars-input_size, steps):
    x.append(content[i:i+input_size])
    y.append(content[i+input_size])

for i in range(10):
    print(f"{x[i]} -> {y}")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

